In [249]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from random import random, choice, randint
from functools import reduce
from collections import namedtuple
from copy import copy
from math import floor, exp 

def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    #seed(random())
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets


In [250]:
NUM_SETS = 1000
NUM_POINTS = 1000
DENSITY = .3

SETS = make_set_covering_problem(NUM_SETS, NUM_POINTS, DENSITY)
SETS = SETS.toarray()

### In my idea a set that led to not an improvement when removed from solution need to be switched with less probability
### The opposite for a set that led to not an improvement when put in solution

In [251]:
survived = [ 0 for _ in range(NUM_SETS)]
dead = [ 0 for _ in range(NUM_SETS)]

In [ ]:
def f(x): #FROM ARRAY OF BOOLEAN TO ARRAY OF 0,1
    if x:
        return 1
    else:
        return 0

### How many sets cover i-th segment

In [ ]:
scores = list(map(lambda x : 1/x,
        reduce(
            lambda x,y : x+y,
            [list(map(f,SETS[i])) for i in range(NUM_SETS)],
            np.array([False for _ in range(NUM_POINTS)]),
    )))

In [252]:
def fitness(state): #For less dense tiles
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost, valuta_sol(state)

def fitness2(state): #For more dense tiles
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost

def valuta_sol(state): #Based on how many less covered segments my sol covers
    return sum([scores[i] for i,t in enumerate(state) if t])

def find_index(state, swap_in):
    not_found = True
    devo_effettuare_swap_in = random() <= swap_in
    while(not_found):
        index = randint(0, NUM_POINTS - 1)
        if(not devo_effettuare_swap_in and any(state)):
            if(state[index]): #SWAP OUT
                if(survived[index] == 0):
                    not_found = False
                else:
                    not_found = random() >= 1/survived[index]
        else:   #SWAP IN
            if(dead[index] == 0):
                not_found = False
            else:
                not_found = random() >= 1/dead[index]
    return index


def tweak(state, temperature, swap_in, n_evaluation):
    act_best = copy(state)
    act_fitness_best = fitness(state)
    improved = False
    for _ in range(n_evaluation):
        tmp_state = copy(state)
        indexes = list()
        for _ in range(floor(temperature)):
            index = find_index(tmp_state, swap_in)
            tmp_state[index] = not tmp_state[index]
            indexes.append(index)
        tmp_fitness = fitness(tmp_state)
        if(tmp_fitness > act_fitness_best):
            act_best = copy(tmp_state)
            act_fitness_best = tmp_fitness
            improved = True
            for i in range(NUM_SETS):
                survived[i] = 0
                dead[i] = 0
        else:
            for index in indexes:
                if tmp_state[index]:
                    dead[index] += 1
                else:
                    survived[index] += 1
            
    return act_best, indexes, improved

In [253]:
original_state = [choice([False]) for _ in range(NUM_SETS)]
current_state = copy(original_state)

In [ ]:
if DENSITY == .3:
    early_termination = 10
    fitness = fitness
    num_gen = 100
else:
    early_termination = 3
    fitness = fitness2
    num_gen = 10    
if NUM_SETS == 100:
    alpha = 2
    n_evaluation = 500 #Number of mutation to evaluate at each step (Steepest step)
elif NUM_SETS == 1000:
    alpha = 4
    n_evaluation = 500 #Number of mutation to evaluate at each step (Steepest step)
elif NUM_SETS == 5000:
    alpha = 5
    n_evaluation = 100

In [254]:
no_improv = 0   #Number of step w/o improving solution
beta = 0        #Parameter for tuning temperature (number of swap at each mutation) 

for step in range(num_gen):
    beta+=1
    if no_improv >= early_termination: #By experience, should be optimized
        break
    if random() <= .1 and fitness(current_state) > (NUM_SETS,-NUM_POINTS,0):    #I want a random restart only if i'm taking all segments
        no_improv = 0
        beta = 0
        current_state = copy(original_state) #Random restart
    temperature = alpha*exp(-0.15*beta)+1
    swap_in = 0.5 #Probability of swapping out or swapping in
    #swap_in = -0.07*step+1+.08*no_improv #Early try of tuning
    new_state, indexes, improved = tweak(current_state, temperature, swap_in, n_evaluation) #Steepest steph
    if improved:
        no_improv = 0
        current_state = new_state
        print(fitness(current_state), step)    
    else:
        no_improv += 1

print("Resolved in ", (step+1)*n_evaluation)



(99, -13)
(100, -14)
(100, -13)
(100, -12)
(100, -11)
(100, -10)
(100, -9)
